### Teraz zastosujemy algorytm uczenia perceptronu dla danych z polskiej giełdy, w tym celu trzeba go nieco zmodyfikować - ograniczymy liczbę obrotów pętli i dodamy warunek zatrzymania, gdy dokładność predykcji jest u nas dość wysoka (tutaj 0.8)

In [1]:
import numpy as np
from random import random
import pandas as pd

In [2]:
def activ(x):
    """Funkcja aktywacji signum."""
    if x > 0 :
        return 1
    else:
        return -1
def perceptron_epochs(x,y, func, ep = 100):
    """
    Implementacja algorytmu perceptronu.
    
    Parametry:
    x - macierz wejść (dane uczące)
    y - wektor oczekiwanych wyników (etykiety)
    func - funkcja aktywacji
    ep - liczba obrotów pętli (domyślnie = 100)
    
    Zwraca:
    w - wektor wag
    theta - wartość biasu (przesunięcia)
    """
    # Inicjalizacja wag i biasu losowymi wartościami
    w = [random() for i in range(x.shape[1])]
    theta = random()
    e = 0
    while e < ep:
        temp = 0
        idx = 0
        
        # Sprawdzamy, które przykłady są klasyfikowane poprawnie
        for i in range(len(x)):
            if func(np.dot(x[i], w) + theta) == y[i]:
                temp += 1
            else:
                idx = i
                break
        # Jeżeli wszystkie przykłady są klasyfikowane poprawnie lub poprawnośc jest >= 0.8 wszystkich przykładów, kończymy
        if temp == len(x) or temp >= 0.8 * len(x):
            break
        
        # Modyfikujemy wagi i bias na podstawie pierwszego błędnie klasyfikowanego przykładu
        w = w + y[idx] * x[idx]
        theta = theta + y[idx]
        e += 1
    return w, theta

In [3]:
# Wczytanie danych z pliku CSV
wig20 = pd.read_csv("wig20_w.csv")

# Obliczenie zwrotów i opóźnień dla danych
wig20['zwroty'] = (wig20['Zamkniecie'] - wig20['Otwarcie']) * 100 / wig20['Otwarcie']
wig20['zam1'] = wig20['Zamkniecie'].shift(1)
wig20['otw1'] = wig20['Otwarcie'].shift(1)
wig20['otw2'] = wig20['Otwarcie'].shift(2)
wig20['otw3'] = wig20['Otwarcie'].shift(3)
wig20['otw4'] = wig20['Otwarcie'].shift(4)
wig20['otw5'] = wig20['Otwarcie'].shift(5)
wig20['maks1'] = wig20['Najwyzszy'].shift(1)
wig20['min1'] = wig20['Najnizszy'].shift(1)

wig20['lag1'] = (wig20['zam1'] - wig20['otw1']) * 100 / wig20['otw1']
wig20['lag2'] = (wig20['zam1'] - wig20['otw2']) * 100 / wig20['otw2']
wig20['lag3'] = (wig20['zam1'] - wig20['otw3']) * 100 / wig20['otw3']
wig20['lag4'] = (wig20['zam1'] - wig20['otw4']) * 100 / wig20['otw4']
wig20['lag5'] = (wig20['zam1'] - wig20['otw5']) * 100 / wig20['otw5']

wig20['zmiana'] = np.where(wig20['zwroty'] > 0, 1, -1)

# Usunięcie pierwszych pięciu wierszy z ramki danych (nie mamy dla nich wszystkich wartości zwrotów)
wig20 = wig20.iloc[5:]

# Konwersja kolumny 'Data' na typ daty
wig20['Data'] = pd.to_datetime(wig20['Data'], format='%Y-%m-%d')

In [4]:
def alg_test(start_date_tr, end_date_tr, end_date_test, L = ['Data', 'lag1', 'zmiana']):
    
    """
    Funkcja trenująca i testująca perceptron na danych giełdowych.

    Parametry:
    start_date_tr (str): Data początkowa dla zbioru treningowego.
    end_date_tr (str): Data końcowa dla zbioru treningowego.
    end_date_test (str): Data końcowa dla zbioru testowego.
    L (list): Lista kolumn, w której 'Data' musi być pierwszą kolumną, a 'zmiana' ostatnią (domyślnie ['Data', 'lag1', 'zmiana']).

    Zwraca:
    tuple: Maksymalna dokładność dla zbioru treningowego oraz testowego.
    """

    # Utworzenie zbioru treningowego zawierającego dane przed "2004-12-31"
    train = wig20[(wig20['Data'] >= start_date_tr) & (wig20['Data'] <= end_date_tr)][L]

    # Konwersja danych treningowych na macierz i wyodrębnienie X i Y
    matr = train.to_numpy()
    X_train = matr[:, 1:len(L)-1].astype(float)
    Y_train = matr[:, len(L)-1].astype(float)

    # Utworzenie zbioru testowego zawierającego dane po "2004-12-31"
    test = wig20[(wig20['Data'] > end_date_tr) & (wig20['Data'] <= end_date_test)][L]

    # Konwersja danych treningowych na macierz i wyodrębnienie X i Y
    matr = test.to_numpy()
    X_test = matr[:, 1:len(L)-1].astype(float)
    Y_test = matr[:, len(L)-1].astype(float)

    vec_train = []
    vec_test = []
    for i in range(20):
        ##dokładnośc predykcji dla zbioru treningowego
        wagi, theta = perceptron_epochs(X_train,Y_train, activ, ep = 100000)
        Y_pred = []
        for i in range(len(X_train)):
            Y_pred.append(activ(X_train[i].dot(wagi) + theta))
        sum(Y_pred == Y_train)/len(Y_train)
        accuracy = sum(Y_pred == Y_train) / len(Y_train)
        vec_train.append(accuracy)

        ##dokładnośc predykcji dla zbioru testowego
        Y_pred = []                               
        for i in range(len(X_test)):
            Y_pred.append(activ(np.dot(X_test[i], wagi) + theta))
        Y_pred = np.array(Y_pred)
        Y_test = np.array(Y_test)
        accuracy = sum(Y_pred == Y_test) / len(Y_test)
        vec_test.append(accuracy)
    return("Max_accuracy (train):" , max(vec_train), " Max accuracy (test):", max(vec_test))

In [5]:
alg_test('2004-06-20','2004-12-31','2005-02-28', L = ['Data', 'lag1', 'lag2', 'lag3', 'lag4', 'lag5', 'zmiana'])

('Max_accuracy (train):',
 0.6785714285714286,
 ' Max accuracy (test):',
 0.6666666666666666)

In [6]:
alg_test('2004-06-20','2004-12-31','2005-02-28', L = ['Data', 'lag1', 'lag2', 'lag3', 'lag4',  'zmiana'])

('Max_accuracy (train):',
 0.7142857142857143,
 ' Max accuracy (test):',
 0.6666666666666666)

In [7]:
alg_test('2004-06-20','2004-12-31','2005-02-28', L = ['Data', 'lag1', 'lag2', 'lag3', 'lag4', 'maks1','min1','zmiana'])

##zmienne maks1 i min1 w tym przypadku pogorszyly wyniki na zbiorze treningowym

('Max_accuracy (train):',
 0.6428571428571429,
 ' Max accuracy (test):',
 0.6666666666666666)

In [8]:
alg_test('2004-06-20','2004-12-31','2005-02-28', L = ['Data', 'lag1', 'lag2', 'lag3', 'zmiana'])

('Max_accuracy (train):',
 0.6071428571428571,
 ' Max accuracy (test):',
 0.5555555555555556)

In [9]:
alg_test('2004-06-20','2004-12-31','2005-02-28', L = ['Data', 'lag1', 'lag2', 'zmiana'])

('Max_accuracy (train):', 0.5, ' Max accuracy (test):', 0.7777777777777778)

In [17]:
alg_test('2004-06-20','2004-12-31','2005-02-28', L = ['Data', 'lag1', 'zmiana'])

('Max_accuracy (train):',
 0.7142857142857143,
 ' Max accuracy (test):',
 0.6666666666666666)

## Do predykcji na tydzień w przód zmodyfikowałam nieco kod funkcji i tutaj dokładność przewidywań określa nam ilośc poprawnych trafień na 20 obrotów pętli (mamy w końcu tylko 1 rekord w zbiorze testowym)

In [11]:
# Utworzenie zbioru treningowego zawierającego dane przed "2004-12-31"
train = wig20[wig20['Data'] < '2004-12-31'][['Data', 'lag1', 'lag2', 'lag3', 'lag4', 'zmiana']]

# Konwersja danych treningowych na macierz i wyodrębnienie X i Y
matr = train.to_numpy()
X_train = matr[:, 1:5].astype(float)
Y_train = matr[:, 5].astype(float)

# Utworzenie zbioru testowego zawierającego dane po "2004-12-31" i przed "2005-01-07"
test = wig20[(wig20['Data'] >= '2004-12-31') & (wig20['Data'] <= '2005-01-07')][['Data', 'lag1', 'lag2', 'lag3', 'lag4','zmiana']]
# Konwersja danych treningowych na macierz i wyodrębnienie X i Y
matr = test.to_numpy()
X_test = matr[:, 1:5].astype(float)
Y_test = matr[:, 5].astype(float)
print(test)

vec_train = []
vec_test = []
correct_classifications = []
for i in range(20):
    ##dokładnośc predykcji dla zbioru treningowego
    wagi, theta = perceptron_epochs(X_train,Y_train, activ, ep = 100000)
    Y_pred = []
    for i in range(len(X_train)):
        Y_pred.append(activ(X_train[i].dot(wagi) + theta))
    sum(Y_pred == Y_train)/len(Y_train)
    accuracy = sum(Y_pred == Y_train) / len(Y_train)
    vec_train.append(accuracy)

    ##dokładnośc predykcji dla zbioru testowego
    Y_pred = []                               
    for i in range(len(X_test)):
        Y_pred.append(activ(np.dot(X_test[i], wagi) + theta))
    Y_pred = np.array(Y_pred)
    Y_test = np.array(Y_test)
    #print(Y_pred == Y_test)
    correct_classifications.append(Y_pred == Y_test)
    accuracy = sum(Y_pred == Y_test) / len(Y_test)
    vec_test.append(accuracy)
correct_classifications = np.array(correct_classifications)
count_true = np.sum(correct_classifications)
count_false = correct_classifications.size - count_true
print("Na 20 obrotów pętli mamy ", count_true, "poprawnych klasyfikacji")
print("Max_accuracy (train):" , max(vec_train), "\n Max accuracy (test):", max(vec_test))

         Data      lag1      lag2      lag3     lag4  zmiana
33 2005-01-02 -0.630345  2.789237  1.461753  4.61364       1
Na 20 obrotów pętli mamy  17 poprawnych klasyfikacji
Max_accuracy (train): 0.7142857142857143 
 Max accuracy (test): 1.0


## testujemy na miesiąc do przodu na innym okresie czasu na modelu [lag1, lag2, lag3, lag4] 

In [12]:
alg_test('2007-04-30','2007-10-31','2007-11-30', L = ['Data', 'lag1', 'lag2', 'lag3', 'lag4',  'zmiana'])

('Max_accuracy (train):', 0.7692307692307693, ' Max accuracy (test):', 0.5)

## modyfikujemy wielkość zbioru testowego i pokazujemy wzrost dokładności predykcji

In [13]:
alg_test('2004-06-20','2004-12-31','2005-08-28', L = ['Data', 'lag1', 'lag2', 'lag3', 'lag4',  'zmiana'])
##osiem miesiacy

('Max_accuracy (train):', 0.75, ' Max accuracy (test):', 0.5428571428571428)

In [14]:
alg_test('2004-06-20','2004-12-31','2005-04-28', L = ['Data', 'lag1', 'lag2', 'lag3', 'lag4',  'zmiana'])
##cztery miesiace

('Max_accuracy (train):',
 0.7142857142857143,
 ' Max accuracy (test):',
 0.6470588235294118)

In [15]:
alg_test('2004-06-20','2004-12-31','2005-02-28', L = ['Data', 'lag1', 'lag2', 'lag3', 'lag4',  'zmiana'])
##dwa miesiace

('Max_accuracy (train):',
 0.7142857142857143,
 ' Max accuracy (test):',
 0.6666666666666666)

In [16]:
alg_test('2004-06-20','2004-12-31','2005-01-31', L = ['Data', 'lag1', 'lag2', 'lag3', 'lag4',  'zmiana'])
##miesiac

('Max_accuracy (train):', 0.75, ' Max accuracy (test):', 0.8)